### Homework: going neural (6 pts)

We've checked out statistical approaches to language models in the last notebook. Now let's go find out what deep learning has to offer.

<img src='https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/expanding_mind_lm_kn_3.png' width=300px>

We're gonna use the same dataset as before, except this time we build a language model that's character-level, not word level. Before you go:
* If you haven't done seminar already, use `seminar.ipynb` to download the data.
* This homework uses Pytorch v1.x: this is [how you install it](https://pytorch.org/get-started/locally/); and that's [how you use it](https://github.com/yandexdataschool/Practical_RL/tree/9f89e98d7df7ad47f5d6c85a70a38283e06be16a/week04_%5Brecap%5D_deep_learning).

In [1]:
import sys
sys.path.append("..")
from utils import map
from plotter import History, plot_metrics
from time import sleep

In [2]:
from importlib import reload
import plotter
reload(plotter)

<module 'plotter' from '/mnt/10A6-8381/books/SDA/3/00_future/01_NLP/nlp_course/week03_lm/../plotter.py'>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Working on character level means that we don't need to deal with large vocabulary or missing words. Heck, we can even keep uppercase words in text! The downside, however, is that all our sequences just got a lot longer.

However, we still need special tokens:
* Begin Of Sequence  (__BOS__) - this token is at the start of each sequence. We use it so that we always have non-empty input to our neural network. $P(x_t) = P(x_1 | BOS)$
* End Of Sequence (__EOS__) - you guess it... this token is at the end of each sequence. The catch is that it should __not__ occur anywhere else except at the very end. If our model produces this token, the sequence is over.


In [4]:
from functools import wraps

def pass_device(func):
    @wraps(func)
    def inner(*args, **kwargs):
        for i, arg in enumerate(args):
            if isinstance(arg, str) and arg in ["cpu", "cuda"]:
                kwargs["device"] = arg
                args = args[:i] + args[i + 1:]
                break
            
        kwargs["device"] = kwargs.get("device", device)
        return func(*args, **kwargs)
    return inner

In [5]:
BOS, EOS = ' ', '\n'

data = pd.read_json("./arxivData.json")
lines = data.apply(lambda row: (row['title'] + ' ; ' + row['summary'])[:512], axis=1) \
            .apply(lambda line: BOS + line.replace(EOS, ' ') + EOS) \
            .tolist()

# if you missed the seminar, download data here - https://yadi.sk/d/_nGyU2IajjR9-w

Our next step is __building char-level vocabulary__. Put simply, you need to assemble a list of all unique tokens in the dataset.

In [6]:
# get all unique characters from lines (including capital letters and symbols)
tokens = list(set().union(*lines))

tokens = sorted(tokens)
n_tokens = len(tokens)
print ('n_tokens = ',n_tokens)
assert 100 < n_tokens < 150
assert BOS in tokens, EOS in tokens

n_tokens =  136


We can now assign each character with it's index in tokens list. This way we can encode a string into a torch-friendly integer vector.

In [7]:
# dictionary of character -> its identifier (index in tokens list)
token_to_id = dict(zip(tokens, range(len(tokens))))

In [8]:
assert len(tokens) == len(token_to_id), "dictionaries must have same size"
for i in range(n_tokens):
    assert token_to_id[tokens[i]] == i, "token identifier must be it's position in tokens list"

print("Seems alright!")

Seems alright!


Our final step is to assemble several strings in a integet matrix `[batch_size, text_length]`. 

The only problem is that each sequence has a different length. We can work around that by padding short sequences with extra _EOS_ or cropping long sequences. Here's how it works:

In [9]:
def to_matrix(lines, max_len=None, pad=token_to_id[EOS], dtype=np.int64):
    """Casts a list of lines into torch-digestable matrix"""
    max_len = max_len or max(map(len, lines))
    lines_ix = np.full([len(lines), max_len], pad, dtype=dtype)
    for i in range(len(lines)):
        line_ix = list(map(token_to_id.get, lines[i][:max_len]))
        lines_ix[i, :len(line_ix)] = line_ix
    return lines_ix

In [10]:
#Example: cast 4 random names to matrices, pad with zeros
dummy_lines = [
    ' abc\n',
    ' abacaba\n',
    ' abc1234567890\n',
]
print(to_matrix(dummy_lines))



[[ 1 66 67 68  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 66 67 66 68 66 67 66  0  0  0  0  0  0  0]
 [ 1 66 67 68 18 19 20 21 22 23 24 25 26 17  0]]


### Neural Language Model (2 points including training)

Just like for N-gram LMs, we want to estimate probability of text as a joint probability of tokens (symbols this time).

$$P(X) = \prod_t P(x_t \mid x_0, \dots, x_{t-1}).$$ 

Instead of counting all possible statistics, we want to train a neural network with parameters $\theta$ that estimates the conditional probabilities:

$$ P(x_t \mid x_0, \dots, x_{t-1}) \approx p(x_t \mid x_0, \dots, x_{t-1}, \theta) $$


But before we optimize, we need to define our neural network. Let's start with a fixed-window (aka convolutional) architecture:

<img src='https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/fixed_window_lm.jpg' width=400px>


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
from torch_utils import Apply

class FixedWindowLanguageModel(nn.Module):
    def __init__(self, n_tokens=n_tokens, emb_size=16, hid_size=64):
        """ 
        A fixed window model that looks on at least 5 previous symbols.
        
        Note: fixed window LM is effectively performing a convolution over a sequence of words.
        This convolution only looks on current and previous words.
        Such convolution can be represented as a sequence of 2 operations:
        - pad input vectors by {strides * (filter_size - 1)} zero vectors on the "left", do not pad right
        - perform regular convolution with {filter_size} and {strides}
        
        - If you're absolutely lost, here's a hint: use nn.ZeroPad2d((NUM_LEADING_ZEROS, 0, 0, 0))
          followed by a nn.Conv1d(..., padding=0). And yes, its okay that padding is technically "2d".
        """
        super().__init__() # initialize base class to track sub-layers, trainable variables, etc.
        
        self._emb_size = emb_size
        self._hid_size = hid_size
        self._filter_size = 5
        self._stride = 1
        self._hid2 = 256
        
        self.embedding = nn.Sequential(
            nn.ZeroPad2d((self._stride * (self._filter_size - 1), 0, 0, 0)),
            nn.Embedding(n_tokens, emb_size)
        )
        self.const_window_layer = nn.Sequential(
            Apply(torch.permute, (0, 2, 1)),
            nn.Conv1d(emb_size, hid_size, self._filter_size),
            Apply(torch.permute, (0, 2, 1)),
        )
        self.mlp = nn.Sequential(
            nn.Linear(hid_size, self._hid2),
            nn.ReLU(),
            nn.Dropout1d(.3),
            Apply(torch.permute, (0, 2, 1)),
            nn.BatchNorm1d(self._hid2),
            Apply(torch.permute, (0, 2, 1)),
            nn.Linear(self._hid2, n_tokens)
        )
        
        self.layers = nn.Sequential(self.embedding, self.const_window_layer, self.mlp)
    
    def forward(self, X):
        """
        compute language model logits given input tokens
        :param input_ix: batch of sequences with token indices, tensor: int32[batch_size, sequence_length]
        :returns: pre-softmax linear outputs of language model [batch_size, sequence_length, n_tokens]
            these outputs will be used as logits to compute P(x_t | x_0, ..., x_{t - 1})
            
        :note: that convolutions operate with tensors of shape [batch, channels, length], while linear layers
         and *embeddings* use [batch, length, channels] tensors. Use tensor.permute(...) to adjust shapes.
         
        # output tensor should be of shape [batch_size, sequence_length, n_tokens]
        """
        return self.layers(X)
    
    @pass_device
    def get_possible_next_tokens(self, prefix=BOS, temperature=1.0, max_len=100, *, device):
        """ :returns: probabilities of next token, dict {token : prob} for all tokens """
        device
        
        prefix_ix = torch.as_tensor(to_matrix([prefix]), dtype=torch.int64, device=device)
        with torch.no_grad():
            probs = torch.softmax(self(prefix_ix)[0, -1], dim=-1).cpu().numpy()  # shape: [n_tokens]
        return dict(zip(tokens, probs))
        

In [14]:
dummy_model = FixedWindowLanguageModel()
dummy_model.eval()

dummy_input_ix = torch.as_tensor(to_matrix(dummy_lines))
dummy_logits = dummy_model(dummy_input_ix)

print('Weights:', tuple(name for name, _ in dummy_model.named_parameters()))

Weights: ('embedding.1.weight', 'const_window_layer.1.weight', 'const_window_layer.1.bias', 'mlp.0.weight', 'mlp.0.bias', 'mlp.4.weight', 'mlp.4.bias', 'mlp.6.weight', 'mlp.6.bias')


In [15]:
assert isinstance(dummy_logits, torch.Tensor)
assert dummy_logits.shape == (len(dummy_lines), max(map(len, dummy_lines)), n_tokens), "please check output shape"
assert np.all(np.isfinite(dummy_logits.data.cpu().numpy())), "inf/nan encountered"
assert not np.allclose(dummy_logits.data.cpu().numpy().sum(-1), 1), "please predict linear outputs, don't use softmax (maybe you've just got unlucky)"

In [16]:
# test for lookahead
dummy_input_ix_2 = torch.as_tensor(to_matrix([line[:3] + 'e' * (len(line) - 3) for line in dummy_lines]))
dummy_logits_2 = dummy_model(dummy_input_ix_2)

assert torch.allclose(dummy_logits[:, :3], dummy_logits_2[:, :3]), "your model's predictions depend on FUTURE tokens. " \
    " Make sure you don't allow any layers to look ahead of current token." \
    " You can also get this error if your model is not deterministic (e.g. dropout). Disable it for this test."

We can now tune our network's parameters to minimize categorical crossentropy over training dataset $D$:

$$ L = {\frac1{|D|}} \sum_{X \in D} \sum_{x_i \in X} - \log p(x_t \mid x_1, \dots, x_{t-1}, \theta) $$

As usual with with neural nets, this optimization is performed via stochastic gradient descent with backprop.

One can also note that minimizing crossentropy is equivalent to
minimizing model __perplexity__, KL-divergence or maximizng log-likelihood.

In [17]:
def compute_mask(input_ix, eos_ix=token_to_id[EOS]):
    """ compute a boolean mask that equals "1" until first EOS (including that EOS) """
    return F.pad(torch.cumsum(input_ix == eos_ix, dim=-1)[..., :-1] < 1, pad=(1, 0, 0, 0), value=True)

print('matrix:\n', dummy_input_ix.numpy())
print('mask:', compute_mask(dummy_input_ix).to(torch.int32).cpu().numpy())
print('lengths:', compute_mask(dummy_input_ix).sum(-1).cpu().numpy())

matrix:
 [[ 1 66 67 68  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 66 67 66 68 66 67 66  0  0  0  0  0  0  0]
 [ 1 66 67 68 18 19 20 21 22 23 24 25 26 17  0]]
mask: [[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
lengths: [ 5  9 15]


In [18]:
@pass_device
def compute_loss(model, input_ix, *, device):
    """
    :param model: language model that can compute next token logits given token indices
    :param input ix: int32 matrix of tokens, shape: [batch_size, length]; padded with eos_ix
    :returns: scalar loss function, mean crossentropy over non-eos tokens
    """
    input_ix = torch.as_tensor(input_ix, dtype=torch.int64, device=device)
    

    logits = model(input_ix[:, :-1])
    reference_answers = input_ix[:, 1:]

    # Your task: implement loss function as per formula above
    # your loss should only be computed on actual tokens, excluding padding
    # predicting actual tokens and first EOS do count. Subsequent EOS-es don't
    # you may or may not want to use the compute_mask function from above.
    
    logs = nn.CrossEntropyLoss(reduction='none')(torch.permute(logits, (0, 2, 1)), reference_answers)
    return (logs * compute_mask(input_ix)[:, 1:]).sum(axis=1).mean()
    

In [19]:

loss_1 = compute_loss(dummy_model, to_matrix(dummy_lines, max_len=15), "cpu")
loss_2 = compute_loss(dummy_model, to_matrix(dummy_lines, max_len=16), "cpu")
assert (np.ndim(loss_1) == 0) and (0 < loss_1 < 100), "loss must be a positive scalar"
assert torch.allclose(loss_1, loss_2), 'do not include  AFTER first EOS into loss. '\
    'Hint: use compute_mask. Beware +/-1 errors. And be careful when averaging!'

### Evaluation

You will need two functions: one to compute test loss and another to generate samples. For your convenience, we implemented them both in your stead.

In [20]:
@pass_device
def score_lines(model, dev_lines, batch_size, device):
    """ computes average loss over the entire dataset """
    dev_loss_num, dev_loss_len = 0., 0.
    with torch.no_grad():
        for i in range(0, len(dev_lines), batch_size):
            batch_ix = to_matrix(dev_lines[i: i + batch_size])
            dev_loss_num += compute_loss(model, batch_ix, device).item() * len(batch_ix)
            dev_loss_len += len(batch_ix)
    return {"loss": dev_loss_num / dev_loss_len}

def generate(model, prefix=BOS, temperature=1.0, max_len=100):
    """
    Samples output sequence from probability distribution obtained by model
    :param temperature: samples proportionally to model probabilities ^ temperature
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    model.eval()
    with torch.no_grad():
        while True:
            token_probs = model.get_possible_next_tokens(prefix)
            tokens, probs = zip(*token_probs.items())
            if temperature == 0:
                next_token = tokens[np.argmax(probs)]
            else:
                
                probs = np.array([p ** (1. / temperature) for p in probs])
                probs /= sum(probs)
                next_token = np.random.choice(tokens, p=probs)

            prefix += next_token
            if next_token == EOS or len(prefix) > max_len: break
    model.train()
    return prefix

### Training loop

Finally, let's train our model on minibatches of data

In [21]:
from ipywidgets import Output
import ipywidgets as widgets
from collections import defaultdict

class MyOutput(Output):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.is_displayed = False

_outputs = defaultdict(MyOutput)
def _reprint(*args, html_id, **kwargs):
    output = _outputs[html_id]
    if not output.is_displayed:
        output.is_displayed = True
        display(output)
        
    with output:
        output.clear_output()
        print(*args, **kwargs)
        

In [22]:
from _ast import Call, Module
from typing import Any
from itertools import count
import ast


class PrintTransformer(ast.NodeTransformer):
    def __init__(self) -> None:
        self.count = count()
        super().__init__()
        
    def visit_Call(self, node: Call) -> Call:
        if isinstance(node.func, ast.Name) and node.func.id == "print":
            node.func.id = "_reprint"
            node.keywords.append(ast.keyword(
                arg="html_id",
                value=ast.Constant(next(self.count))
            ))
        
        return super().generic_visit(node)
    
    def __eq__(self, other):
        return self is other

class SoftCtrlCTransformer(ast.NodeTransformer):
    def visit_Module(self, node: Module) -> Module:
        try_ = ast.parse("""
try:
    pass
except KeyboardInterrupt:
    print("interrupted by keyboard")
""")
        try_.body[0].body = node.body
        
        return super().generic_visit(try_)
    
    def __eq__(self, other):
        return self is other

In [23]:
from IPython import get_ipython
from IPython.core.magic import register_cell_magic
from IPython.display import clear_output

ipy = get_ipython()
@register_cell_magic
def reprint(_, cell):
    tr = PrintTransformer()
    _outputs.clear()
    clear_output()
    ipy.ast_transformers.append(tr)
    ipy.run_cell(cell)
    if tr in ipy.ast_transformers:
        ipy.ast_transformers.remove(tr)

@register_cell_magic
def soft_ctrl_c(_, cell):
    tr = SoftCtrlCTransformer()
    _outputs.clear()
    clear_output()
    ipy.ast_transformers.append(tr)
    ipy.run_cell(cell)
    if tr in ipy.ast_transformers:
        ipy.ast_transformers.remove(tr)


In [24]:
from dataclasses import dataclass, field
import plotly
import plotly.graph_objects as go
import warnings

from dataclasses import dataclass, field
from pathlib import Path
from math import ceil

import plotly.express as px
import numpy as np
from plotly.subplots import make_subplots

@dataclass
class Plotter:
    metrics: list[str] = field(default_factory=list)
    titles: list[str] = field(default_factory=list)
    height: int = 600
    width: int = 1000
    plot_cols: int = 3
    path: Path | str | None = None
    bound_history: History | None = None
    # custom_range: dict[str, tuple[int, int]] = field(default_factory=dict)
    
    def __post_init__(self):
        if self.path is not None:
            self.path = Path(self.path)
            if self.path.exists():
                warnings.warn("Warning...........Message")
            
        self._inited = False
        if not self.titles:
            self.titles = self.metrics
        
        self.fig: plotly.graph_objs._figure.Figure
        self.traces = {}
        self.names = set()
        
    def init_canvas(self):
        plot_cols = min(self.plot_cols, len(self.metrics))
        plot_rows = max(1, ceil(len(self.metrics) / plot_cols))
        self.fig = go.FigureWidget(make_subplots(rows=plot_rows, cols=plot_cols, subplot_titles=self.titles))
        self.fig.update_layout(height=self.height, width=self.width, showlegend=True)
        display(self.fig)
    
    def draw(self, history: History | None = None):
        if history is None:
            history = self.bound_history
            assert history is not None
        
        df_kept, df_dropped = history.as_dfs()
        if not self._inited:
            if not self.metrics:
                self.metrics = self.titles = list(set(df_dropped.columns) - {"phase", "epoch"})
            self.init_canvas()
            self._inited = True
        
        for handle, df in ("kept", df_kept), ("dropped", df_dropped):
            for i, metric in enumerate(self.metrics):
                for trace in px.scatter(df.dropna(subset=metric),
                                        x="epoch", y=metric, color="phase")["data"]:
                    
                    if (handle, metric, trace.name) in self.traces:
                        _trace = self.traces[(handle, metric, trace.name)]
                        # avoid unneeded assignments, for they cause lags
                        if _trace.y.shape != trace.y.shape:
                            _trace.x, _trace.y = trace.x, trace.y
                        trace = _trace
                    else:
                        self.fig.append_trace(
                            trace,
                            row=i // self.plot_cols + 1, col=i % self.plot_cols + 1
                        )
                        self.traces[(handle, metric, trace.name)] = trace = self.fig.data[-1]
                        # remove duplicate legends
                        if trace.name in self.names:
                            trace.update(showlegend=False)
                        else:
                            self.names.add(trace.name)
                            
                    if trace.x.size > 1 and trace.mode != "lines" and handle != "dropped":
                        trace.mode = "lines"
                        
                    if handle == "dropped" and ("kept", metric, trace.name) in self.traces:
                        kept = self.traces[("kept", metric, trace.name)]
                        y = np.full_like(trace.y, kept.y[0])
                        
                        if not all(trace.y == y):
                            trace.customdata = trace.y[:, None]
                            trace.y = y
                            trace.hovertemplate = trace.hovertemplate.replace(
                                "%{y}",
                                "%{customdata[0]:.2f}" + f"({history.drop_query})"
                            )
                            trace.mode = "markers"
                            trace.marker["symbol"] = "star"
                            trace.marker["size"] = 10
                            trace.marker["color"] = kept.marker["color"]
                    
        if self.path is not None:
            self.fig.write_image(self.path)
            self.fig.write_html(str(self.path) + ".html")
            
    def draw_no_widget(self):
        return plotly.io.from_json(self.fig.to_json())
    

In [25]:
def display_no_widgets(plotter):
    clear_output()
    display(plotter.draw_no_widget())
    print("\n".join(output.outputs[0]["text"] for output in _outputs.values() if output.outputs))
        

In [26]:
from sklearn.model_selection import train_test_split
train_lines, dev_lines = train_test_split(lines, test_size=0.25, random_state=42)

batch_size = 256
score_dev_every = 250
history = History()
model = FixedWindowLanguageModel().to(device)
opt = torch.optim.Adam(model.parameters())

train_history, dev_history = [], []

# hint: if you ever wanted to switch to cuda, do it now.


# score untrained model
dev_history.append((0, score_lines(model, dev_lines, batch_size, device)))
history.push_epoch({}, dev_history[0][1])
print("Sample before training:", generate(model, 'Bridging'))

/home/photon/anaconda3/envs/NLP/lib/python3.11/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,


Sample before training: Bridgingβśρ3<UXsα<ö~εæÉcρá|bÉ"qêb{IτiVvzh_xêγ



In [27]:
%%soft_ctrl_c
%%reprint

from random import sample
from tqdm import trange

plotter = Plotter(bound_history=history)
model.train()

for i in trange(len(history), 5000):
    batch = to_matrix(sample(train_lines, batch_size))
    
    loss_i = compute_loss(model, batch)
    
    opt.zero_grad()
    loss_i.backward()
    opt.step()
        
    train_history.append((i, loss_i.item()))
    history.push_epoch({"loss": loss_i.item()}, {})
    
    if (i + 1) % score_dev_every == 0:
        dev_history.append((i, score_lines(model, dev_lines, batch_size, device)))
        history.val[-1] = dev_history[-1][1]
        
    if (i + 1) % 50 == 0:
        plotter.draw()
        
        print("Generated examples (tau=0.5):")
        print("\n".join(generate(model, temperature=0.5) for _ in range(3)))
    
clear_output()

  1%|          | 46/4999 [00:01<02:56, 28.11it/s]

FigureWidget({
    'data': [],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'loss',
                                'x': 0.5,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'}],
               'height': 600,
               'showlegend': True,
               'template': '...',
               'width': 1000,
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0]}}
})

MyOutput()

MyOutput()

 80%|████████  | 4016/4999 [02:54<00:42, 23.06it/s]


MyOutput()

In [28]:
display_no_widgets(plotter)

Generated examples (tau=0.5):

 A not on the maching and a networks are the best of collection of the point generation of higher res
 A Ceally content improve the tasks of the sincrease learning the context are come independent set of
 Parameters the distic and content parameter the searcheds to the achieve sented and with a special c

interrupted by keyboard



In [29]:
assert np.mean(train_history[:10], axis=0)[1] > np.mean(train_history[-10:], axis=0)[1], "The model didn't converge."
print("Final dev loss:", dev_history[-1][-1])

for i in range(10):
    print(generate(model, temperature=0.5))

Final dev loss: {'loss': 937.2470221393864}
 Semantics interarchitecture the efficient a ronotony and for neural network for the consider to tran
 Randon segmentation identificial convoluting the propose a structs in via using an eare is and heter
 The sample attent in the translation in   symmetric programming and symmetric strategy effect the or
 Features the distribution language of sparse a strong perture of the deep neural network or demonstr
 Contrassing ten-level on a better data time of non-can be approach for contour (Segmentation an emer
 Aser and on supervised form scalable of the densing the reconstraining procested the describer atter
 Deep natural common to achieve the encode to access of the sample a structure search as this predict
 A rate a recent from the learning or descrease of the attent of the system linear results a logic in
 Given point noness in states action of clustering also the into the sentence and to a distance on ea
 Detect to imple-consistering to the p

### RNN Language Models (3 points including training)

Fixed-size architectures are reasonably good when capturing short-term dependencies, but their design prevents them from capturing any signal outside their window.

We can mitigate this problem by using a __recurrent neural network__:

$$ h_0 = \vec 0 ; \quad h_{t+1} = RNN(x_t, h_t) $$

$$ p(x_t \mid x_0, \dots, x_{t-1}, \theta) = dense_{softmax}(h_{t-1}) $$

Such model processes one token at a time, left to right, and maintains a hidden state vector between them. Theoretically, it can learn arbitrarily long temporal dependencies given large enough hidden size.

<img src='https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/rnn_lm.jpg' width=480px>

In [50]:
class RNNLanguageModel(nn.Module):
    def __init__(self, n_tokens=n_tokens, emb_size=16, hid_size=256):
        """ 
        Build a recurrent language model.
        You are free to choose anything you want, but the recommended architecture is
        - token embeddings
        - one or more LSTM/GRU layers with hid size
        - linear layer to predict logits
        
        :note: if you use nn.RNN/GRU/LSTM, make sure you specify batch_first=True
         With batch_first, your model operates with tensors of shape [batch_size, sequence_length, num_units]
         Also, please read the docs carefully: they don't just return what you want them to return :)
        """
        super().__init__() # initialize base class to track sub-layers, trainable variables, etc.
        
        self._lstm_layers = 3
        self._hid2 = 256
        
        self.emb = nn.Embedding(n_tokens, emb_size)
        self.rnn = nn.GRU(emb_size, hid_size, num_layers=self._lstm_layers, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(hid_size, self._hid2),
            nn.ReLU(),
            nn.Dropout1d(.3),
            Apply(torch.permute, (0, 2, 1)),
            nn.BatchNorm1d(self._hid2),
            Apply(torch.permute, (0, 2, 1)),
            nn.Linear(self._hid2, n_tokens)
        )
    
    def forward(self, X):
        """
        compute language model logits given input tokens
        :param input_ix: batch of sequences with token indices, tensor: int32[batch_size, sequence_length]
        :returns: pre-softmax linear outputs of language model [batch_size, sequence_length, n_tokens]
            these outputs will be used as logits to compute P(x_t | x_0, ..., x_{t - 1})
        """
        X = self.emb(X)
        X = self.rnn(X)[0]
        X = self.mlp(X)
        return X
    
    def get_possible_next_tokens(self, prefix=BOS, temperature=1.0, max_len=100):
        """ :returns: probabilities of next token, dict {token : prob} for all tokens """
        prefix_ix = torch.as_tensor(to_matrix([prefix]), dtype=torch.int64, device=device)
        with torch.no_grad():
            probs = torch.softmax(self(prefix_ix)[0, -1], dim=-1).cpu().numpy()  # shape: [n_tokens]
        return dict(zip(tokens, probs))
        

In [31]:
model = RNNLanguageModel()
model.eval()

dummy_input_ix = torch.as_tensor(to_matrix(dummy_lines))
dummy_logits = model(dummy_input_ix)

assert isinstance(dummy_logits, torch.Tensor)
assert dummy_logits.shape == (len(dummy_lines), max(map(len, dummy_lines)), n_tokens), "please check output shape"
assert not np.allclose(dummy_logits.cpu().data.numpy().sum(-1), 1), "please predict linear outputs, don't use softmax (maybe you've just got unlucky)"
print('Weights:', tuple(name for name, w in model.named_parameters()))

Weights: ('emb.weight', 'rnn.weight_ih_l0', 'rnn.weight_hh_l0', 'rnn.bias_ih_l0', 'rnn.bias_hh_l0', 'mlp.0.weight', 'mlp.0.bias', 'mlp.4.weight', 'mlp.4.bias', 'mlp.6.weight', 'mlp.6.bias')


In [32]:
# test for lookahead
dummy_input_ix_2 = torch.as_tensor(to_matrix([line[:3] + 'e' * (len(line) - 3) for line in dummy_lines]))
dummy_logits_2 = model(dummy_input_ix_2)

assert torch.allclose(dummy_logits[:, :3], dummy_logits_2[:, :3]), "your model's predictions depend on FUTURE tokens. " \
    " Make sure you don't allow any layers to look ahead of current token." \
    " You can also get this error if your model is not deterministic (e.g. dropout). Disable it for this test."

### RNN training

Our RNN language model should optimize the same loss function as fixed-window model. But there's a catch. Since RNN recurrently multiplies gradients through many time-steps, gradient values may explode, [ruining](https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/nan.jpg) your model.
The common solution to that problem is to clip gradients either [individually](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/clip_by_value) or [globally](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/clip_by_global_norm).

Your task here is to implement the training code that minimizes the loss function. If you encounter large loss fluctuations during training, please add [gradient clipping](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html) using urls above. But its **not necessary** to use gradient clipping if you don't need it.

_Note: gradient clipping is not exclusive to RNNs. Convolutional networks with enough depth often suffer from the same issue._

In [89]:
batch_size = 255
score_dev_every = 250
train_history, dev_history = [], []
history = History()

model = RNNLanguageModel().to(device)
opt = torch.optim.Adam(model.parameters(), lr=5e-4)

# score untrained model
dev_history.append((0, score_lines(model, dev_lines, batch_size, device)))
history.push_epoch({}, dev_history[0][1])
print("Sample before training:", generate(model, 'Bridging'))
torch.cuda.empty_cache()

Sample before training: BridgingJâtwüu B`[sŁbőw0@3u{[BâÉ{σ*μkWäeuYwÖgP@æN`&°Q=lõçV=KC@pNãoŁ@!%Kçup5IaZ?xŁőxc_X!j#Öbνlsy#ś]áz9


In [91]:
%%soft_ctrl_c
%%reprint

from random import sample
from tqdm import trange

plotter = Plotter(bound_history=history, width=800)
model.train()

for i in trange(len(history), 5000):
    batch = to_matrix(sample(train_lines, batch_size))
    
    loss_i = compute_loss(model, batch)
    
    opt.zero_grad()
    loss_i.backward()
    opt.step()
        
    train_history.append((i, loss_i.item()))
    history.push_epoch({"loss": loss_i.item()}, {})
    
    if (i + 1) % score_dev_every == 0:
        dev_history.append((i, score_lines(model, dev_lines, batch_size, device)))
        history.val[-1] = dev_history[-1][1]
        
    if (i + 1) % 50 == 0:
        plotter.draw()
        
        print("Generated examples (tau=0.5):")
        print("\n".join(generate(model, temperature=0.5) for _ in range(3)))


  1%|          | 48/4999 [00:07<12:54,  6.40it/s]

FigureWidget({
    'data': [],
    'layout': {'annotations': [{'font': {'size': 16},
                                'showarrow': False,
                                'text': 'loss',
                                'x': 0.5,
                                'xanchor': 'center',
                                'xref': 'paper',
                                'y': 1.0,
                                'yanchor': 'bottom',
                                'yref': 'paper'}],
               'height': 600,
               'showlegend': True,
               'template': '...',
               'width': 800,
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0]}}
})

MyOutput()

MyOutput()

100%|██████████| 4999/4999 [14:42<00:00,  5.67it/s]  


In [157]:
display_no_widgets(plotter)

Generated examples (tau=0.5):

 A Similarity of Neural Network for Correlation and   Cartional Activation   from a Single   Processi
 Relational Saliency for Convex   Engineering ; One of the proposed setting where the number of all c
 Restricted Physics with Satellation of   Depth   Learning Approach ; Finite computational regression



In [ ]:
assert np.mean(train_history[:10], axis=0)[1] > np.mean(train_history[-10:], axis=0)[1], "The model didn't converge."
print("Final dev loss:", dev_history[-1][-1])
for i in range(10):
    print(generate(model, temperature=0.5))

### Alternative sampling strategies (1 point)

So far we've sampled tokens from the model in proportion with their probability.
However, this approach can sometimes generate nonsense words due to the fact that softmax probabilities of these words are never exactly zero. This issue can be somewhat mitigated with sampling temperature, but low temperature harms sampling diversity. Can we remove the nonsense words without sacrificing diversity? __Yes, we can!__ But it takes a different sampling strategy.

__Top-k sampling:__ on each step, sample the next token from __k most likely__ candidates from the language model.

Suppose $k=3$ and the token probabilities are $p=[0.1, 0.35, 0.05, 0.2, 0.3]$. You first need to select $k$ most likely words and set the probability of the rest to zero: $\hat p=[0.0, 0.35, 0.0, 0.2, 0.3]$ and re-normalize: 
$p^*\approx[0.0, 0.412, 0.0, 0.235, 0.353]$.

__Nucleus sampling:__ similar to top-k sampling, but this time we select $k$ dynamically. In nucleous sampling, we sample from top-__N%__ fraction of the probability mass.

Using the same  $p=[0.1, 0.35, 0.05, 0.2, 0.3]$ and nucleous N=0.9, the nucleous words consist of:
1. most likely token $w_2$, because $p(w_2) < N$
2. second most likely token $w_5$, $p(w_2) + p(w_5) = 0.65 < N$
3. third most likely token $w_4$ because $p(w_2) + p(w_5) + p(w_4) = 0.85 < N$

And thats it, because the next most likely word would overflow: $p(w_2) + p(w_5) + p(w_4) + p(w_1) = 0.95 > N$.

After you've selected the nucleous words, you need to re-normalize them as in top-k sampling and generate the next token.

__Your task__ is to implement nucleus sampling variant and see if its any good.

In [ ]:
torch.argsort?

In [83]:
def generate_nucleus(model, prefix=BOS, nucleus=0.9, max_len=100):
    """
    Generate a sequence with nucleous sampling
    :param prefix: a string containing space-separated previous tokens
    :param nucleus: N from the formulae above, N \in [0, 1]
    :param max_len: generate sequences with at most this many tokens, including prefix
    
    :note: make sure that nucleous always contains at least one word, even if p(w*) > nucleus
    
    """
    model.eval()
    while True:
        token_probs = model.get_possible_next_tokens(prefix)
        tokens, probs = map(np.array, zip(*token_probs.items()))
        
        indices = probs.argsort()[::-1]
        mask = probs[indices].cumsum() < nucleus
        mask[0] = True
        
        probs = probs[indices[mask]]
        next_token = np.random.choice(tokens[indices[mask]], p=probs / probs.sum())
        
        prefix += next_token
        if next_token == EOS or len(prefix) > max_len: break
    model.train()
    return prefix

In [88]:
for i in range(10):
    print(generate_nucleus(model, nucleus=.7))

 Renging proposed to training algorithm has an approach to real with and a new as time convolutional 
 A image interpretation in many important techniques of computer for manifold to perform of might as 
 Senset Peration of the terms and solution method to relevant interface of account problem of subotic
 Constraint from Agential Temporal Machine Method efficient   interpretation systems and deplain the 
 Contensition process such as a constraint the analysis of the ense the non-decommender in the matrix
 The tocument of the matrix for distribution and a structure each set is a gradient approach to train
 Real-bot also from a color of reconstruction of the corresponding of a structure relations of convol
 Tooup examinent designed to derive a classification tasks with an engined in a time that eneristic c
 Loint Training Similarity control techniques and converse of matching terms are training of the enab
 Tevel Decomposition of the model is a service examples of all deep learning probl

### Bonus quest I: Beam Search (2 pts incl. samples)

At times, you don't really want the model to generate diverse outputs as much as you want a __single most likely hypothesis.__ A single best translation, most likely continuation of the search query given prefix, etc. Except, you can't get it. 

In order to find the exact most likely sequence containing 10 tokens, you would need to enumerate all $|V|^{10}$ possible hypotheses. In practice, 9 times out of 10 you will instead find an approximate most likely output using __beam search__.

Here's how it works:
0. Initial `beam` = [prefix], max beam_size = k
1. for T steps:
2. ` ... ` generate all possible next tokens for all hypotheses in beam, formulate `len(beam) * len(vocab)` candidates
3. ` ... ` select beam_size best for all candidates as new `beam`
4. Select best hypothesis (-es?) from beam

In [92]:
from IPython.display import HTML
# Here's what it looks like:
!wget -q https://raw.githubusercontent.com/yandexdataschool/nlp_course/2020/resources/beam_search.html
HTML("beam_search.html")

In [125]:
def generate_beamsearch(model, prefix=BOS, beam_size=4, length=5):
    model.eval()
    
    beam = [prefix]
    for _ in range(length):
        new_seqs = {(seq, token) : proba
                    for seq in beam
                    for token, proba in model.get_possible_next_tokens(seq).items()
                    if seq[-1] != EOS}
        beam = sorted(new_seqs, key=new_seqs.get, reverse=True)[:beam_size]
        beam = [seq + token for seq, token in beam]
        
    model.train()
    return beam
    

In [129]:
generate_beamsearch(model, prefix=' deep ', beam_size=4, length=10)

[' deep convolutio',
 ' deep neural net',
 ' deep reinforcem',
 ' deep learning a']

In [ ]:
# check it out: which beam size works best?
# find at least 5 prefixes where beam_size=1 and 8 generates different sequences

In [156]:
for i in range(1, 6):
    prefix = "!" * i

    beam_8 = generate_beamsearch(model, prefix=prefix, beam_size=8, length=2)
    beam_1 = generate_beamsearch(model, prefix=prefix, beam_size=1, length=2)
    assert beam_1[0] not in beam_8
    print(f"{prefix=}:\n{beam_1=}\n{beam_8=}\n")

prefix='!':
beam_1=['! T']
beam_8=['!di', '!ad', '!ma', '!A ', '!mi', '!ab', '!mo', '!Mu']

prefix='!!':
beam_1=['!! T']
beam_8=['!!1 ', '!!3 ', '!!2 ', '!!M ', '!!Ma', '!!2e', '!!{e', '!!Mu']

prefix='!!!':
beam_1=['!!! T']
beam_8=['!!!1 ', '!!!E ', '!!!3 ', '!!!M ', '!!!{ ', '!!!2 ', '!!!2e', '!!!Ma']

prefix='!!!!':
beam_1=['!!!! a']
beam_8=['!!!!1 ', '!!!!M ', '!!!!{ ', '!!!!E ', '!!!!2 ', '!!!!3 ', '!!!!T ', '!!!!11']

prefix='!!!!!':
beam_1=['!!!!! a']
beam_8=['!!!!!M ', '!!!!!{ ', '!!!!!E ', '!!!!!2 ', '!!!!!1 ', '!!!!!T ', '!!!!!3 ', '!!!!!11']



[' A Semantic']

In [130]:
generate_nucleus(model, nucleus=0)

' A Semantic Sensor   for the analysis of the algorithm for semantic segmentation is a complete and ef'

### Bonus quest II: Ultimate Language Model (2+ pts)

So you've learned the building blocks of neural language models, you can now build the ultimate monster:  
* Make it char-level, word level or maybe use sub-word units like [bpe](https://github.com/rsennrich/subword-nmt);
* Combine convolutions, recurrent cells, pre-trained embeddings and all the black magic deep learning has to offer;
  * Use strides to get larger window size quickly. Here's a [scheme](https://storage.googleapis.com/deepmind-live-cms/documents/BlogPost-Fig2-Anim-160908-r01.gif) from google wavenet.
* Train on large data. Like... really large. Try [1 Billion Words](http://www.statmt.org/lm-benchmark/1-billion-word-language-modeling-benchmark-r13output.tar.gz) benchmark;
* Use training schedules to speed up training. Start with small length and increase over time; Take a look at [one cycle](https://medium.com/@nachiket.tanksale/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6) for learning rate;

_You are NOT required to submit this assignment. Please make sure you don't miss your deadline because of it :)_